In [ ]:
!pip install -U weaviate-client

In [ ]:
!pip install Spacy

In [ ]:
import sys
!{sys.executable} -m spacy download en

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')


In [4]:
import weaviate
import weaviate.classes as wvc
import requests
import json


In [5]:
client = weaviate.connect_to_wcs(
    cluster_url="",
    auth_credentials=weaviate.auth.AuthApiKey("")
    
)


In [26]:
client.is_ready()

True

In [82]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType
client.collections.create(
    name="DocumentSearch",
    properties=[
        wc.Property(name="DocText", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(),
)

In [83]:
documents = [
    "SQLServer Oracle PowerBI",
    "SQLServer PowerBI Tableau",
    "Tableau Qlikview",
    "Qlikview SnowFlake",
    "Databricks Azure Data Factory"
     ]
DocumentSearch = client.collections.get("DocumentSearch")
with DocumentSearch.batch.dynamic() as batch:
    for i, doc in enumerate(documents):
        properties = {"DocText": doc}
        nlpdoc = nlp(doc)
        vector=list(nlpdoc.vector)
        batch.add_object(properties,vector=vector)

In [84]:
collection = client.collections.get("DocumentSearch")

for item in collection.iterator(
    include_vector=True  # If using named vectors, you can specify ones to include e.g. ['title', 'body'], or True to include all
):
    print(item.properties)
    print(item.vector)

{'docText': 'SQLServer PowerBI Tableau'}
{'default': [-0.7119714617729187, -0.7853469848632812, -0.41044583916664124, 0.27732303738594055, 0.03868657350540161, 0.2699478566646576, 0.02681412361562252, 0.46695324778556824, 0.27723070979118347, 0.20080383121967316, 1.4129939079284668, 0.24312908947467804, -0.918041467666626, 0.12339284271001816, -0.2344948649406433, -0.15067504346370697, -0.7487204670906067, -0.641111433506012, 0.26169803738594055, -0.6851611733436584, -0.26942917704582214, 0.4357844293117523, -0.07673146575689316, -0.34012940526008606, -0.150448739528656, -0.26242515444755554, 0.14353758096694946, 0.9521039128303528, -0.07850106805562973, 0.8715438842773438, -0.7697744965553284, -0.3942698538303375, 1.0104953050613403, -0.1342553347349167, 0.047489702701568604, 1.0428133010864258, -0.1483791619539261, 0.48117196559906006, -0.4045431911945343, 0.698009192943573, -0.33272048830986023, -0.3470068871974945, -0.9996738433837891, 0.4559503495693207, -0.8345726132392883, -0.07

In [87]:
import weaviate.classes.query as wq
query_text = "Databricks"
doc = nlp(query_text)
query_vector= list(doc.vector)
# Get the collection
SkillSet= client.collections.get("DocumentSearch")

# Perform query
response = SkillSet.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["docText"] 
    )  
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P

Qlikview SnowFlake
Distance to query: 0.476

Tableau Qlikview
Distance to query: 0.547

Databricks Azure Data Factory
Distance to query: 0.582

SQLServer Oracle PowerBI
Distance to query: 0.653

SQLServer PowerBI Tableau
Distance to query: 0.683



In [86]:
import weaviate
from weaviate.classes.query import Filter
from weaviate.collections import Collection
from typing import List

# Instantiate your client (not shown). e.g.:
# client = weaviate.connect_to_wcs(...) or
# client = weaviate.connect_to_local()

collection = client.collections.get("DocumentSearch")

# Get property names
property_names = list()
for p in collection.config.get().properties:
    property_names.append(p.name)

query_strings = ["DataBricks:"]


def filter_demo(collection: Collection, property_names: List[str], query_strings: List[str]):
    from weaviate.classes.query import Filter

    for query_string in query_strings:
        print("\n" + "=" * 40 + f"\nHits for: '{query_string}'" + "\n" + "=" * 40)
        for property_name in property_names:
            response = collection.query.fetch_objects(
                filters=Filter.by_property(property_name).equal(query_string),
            )
            if len(response.objects) > 0:
                print(f">> '{property_name}' matches")
                for obj in response.objects:
                    print(obj.properties[property_name])


filter_demo(collection, property_names, query_strings)


Hits for: 'DataBricks:'
>> 'docText' matches
Databricks Azure Data Factory


In [91]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType,VectorDistances
client.collections.create(
    name="DocumentSearch",
    properties=[
        wc.Property(name="DocText", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw( distance_metric=VectorDistances.COSINE),
)

In [92]:
documents = [
    "#SQLServer Oracle PowerBI#",
    "#SQLServer PowerBI Tableau#",
    "#Tableau Qlikview#",
    "#Qlikview SnowFlake#",
    "#Databricks Azure Data Factory#"
     ]
DocumentSearch = client.collections.get("DocumentSearch")
with DocumentSearch.batch.dynamic() as batch:
    for i, doc in enumerate(documents):
        properties = {"DocText": doc}
        nlpdoc = nlp(doc)
        vector=nlpdoc.vector
        batch.add_object(properties,vector=vector)

In [95]:
import weaviate.classes.query as wq
import pandas as pd
query_text = "#Oracle#"
doc = nlp(query_text)
query_vector= list(doc.vector)
# Get the collection
SkillSet= client.collections.get("DocumentSearch")

# Perform query
response = SkillSet.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)
df = pd.DataFrame(columns=['DocText', 'VectorDistance'])

# Inspect the response
for o in response.objects:
    print(
        o.properties["docText"] 
    )  
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P
    x=str({o.metadata.distance})
    df.loc[len(df.index)] = [o.properties["docText"], x] 

display(df)

#Tableau Qlikview#
Distance to query: 0.211

#Qlikview SnowFlake#
Distance to query: 0.255

#SQLServer PowerBI Tableau#
Distance to query: 0.257

#SQLServer Oracle PowerBI#
Distance to query: 0.258

#Databricks Azure Data Factory#
Distance to query: 0.442



,DocText,VectorDistance
0,#Tableau Qlikview#,{0.21081751585006714}
1,#Qlikview SnowFlake#,{0.2545745372772217}
2,#SQLServer PowerBI Tableau#,{0.25669795274734497}
3,#SQLServer Oracle PowerBI#,{0.25762301683425903}
4,#Databricks Azure Data Factory#,{0.44215601682662964}


In [ ]:
import weaviate.classes.config as wc

client.collections.create(
    name="SkillSet",
    properties=[
        wc.Property(name="EmployeeId", data_type=wc.DataType.TEXT),
        wc.Property(name="Skills", data_type=wc.DataType.TEXT),
       
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none()
  
)


In [ ]:
SkillSet = client.collections.get("SkillSet")

In [ ]:
from weaviate.util import generate_uuid5
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

 
# reading csv file 
df = pd.read_csv("F:\\TechManagement\\weviate\\Skills.txt")
df=df.astype(str)
SkillSet = client.collections.get("SkillSet")
for index, row in df.iterrows():
    with SkillSet.batch.dynamic() as batch:
 
        doc = nlp(row['SkillSet'])
        vec = doc.vector
        SkillSet_obj = {
            "employeeId": str(row['EmpId']),
            "skills": str(row['SkillSet']),
        }
        batch.add_object(
            properties=SkillSet_obj,
            vector=vec  # Add the custom vector
            # references=reference_obj  # You can add references here
        )
if len(SkillSet.batch.failed_objects) > 0:
    print(f"Failed to import {len(SkillSet.batch.failed_objects)} objects")
        


In [ ]:
collection = client.collections.get("SkillSet")

for item in collection.iterator(
    include_vector=True  # If using named vectors, you can specify ones to include e.g. ['title', 'body'], or True to include all
):
    print(item.properties)
    print(item.vector)

In [9]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType
client.collections.create(
    name="NameSearch",
    properties=[
        wc.Property(name="PersonName", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(),
)

In [19]:
from faker import Faker  
import spacy
nlp = spacy.load('en_core_web_sm')
import time

t0 = time.time()

fake = Faker()  
DocumentSearch = client.collections.get("NameSearch")
with DocumentSearch.batch.dynamic() as batch:
    for i in range(0, 100000): 
        fakename =fake.name()
        if (i==1):
            print(fakename)  
        properties = {"personName": fakename}
        nlpdoc = nlp(fakename)
        vector=nlpdoc.vector
        batch.add_object(properties,vector=vector)
t1 = time.time()
total = t1-t0
print(total)  

Paula Merritt
553.3135850429535


In [81]:
import weaviate.classes.query as wq
query_text = "Paula Meritt"
import time
doc = nlp(query_text)
query_vector= list(doc.vector)
# Get the collection
SkillSet= client.collections.get("NameSearch")


t0 = time.time()
# Perform query
response = SkillSet.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["personName"] 
    )  
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P
t1 = time.time()
total = t1-t0
print('Time Taken')
print(total)  

Paula Hines
Distance to query: 0.100

Paula Stevens
Distance to query: 0.101

Paula Stein
Distance to query: 0.114

Paula Merritt
Distance to query: 0.131

Paula Welch
Distance to query: 0.132

Time Taken
0.6708846092224121


In [18]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType
client.collections.create(
    name="NameSearch",
    properties=[
        wc.Property(name="PersonName", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(),
)

In [79]:
import weaviate.classes.query as wq
query_text = "Paula Merrit"
import time
doc = nlp(query_text)
query_vector= list(doc.vector)
# Get the collection
SkillSet= client.collections.get("NameSearch")


t0 = time.time()
# Perform query
response = SkillSet.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=30,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["personName"] 
    )  
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P
t1 = time.time()
total = t1-t0
print('Time Taken')
print(total)  

Paula Thompson
Distance to query: 0.070

Paula Mcdonald
Distance to query: 0.070

Paula Howell
Distance to query: 0.070

Paula Whitney
Distance to query: 0.077

Paula Boyle
Distance to query: 0.077

Paula Soto
Distance to query: 0.080

Paula Guzman
Distance to query: 0.088

Paula Ball
Distance to query: 0.088

Paula Kelley
Distance to query: 0.089

Paula Lambert
Distance to query: 0.089

Paula Barton
Distance to query: 0.089

Paula Frank
Distance to query: 0.090

Anne Wall
Distance to query: 0.092

Anne Thompson
Distance to query: 0.092

Paula Bonilla
Distance to query: 0.092

Matthew Lee
Distance to query: 0.093

Matthew Lee
Distance to query: 0.093

Paula Horton
Distance to query: 0.093

Matthew Riley
Distance to query: 0.094

Pamela Thompson
Distance to query: 0.094

Rickey Rice
Distance to query: 0.094

Paula Wilson
Distance to query: 0.096

Paula Wilson
Distance to query: 0.096

Paula Green
Distance to query: 0.096

Paula Green
Distance to query: 0.096

Paula Payne
Distance to que

In [77]:
import weaviate
from weaviate.classes.query import Filter
from weaviate.collections import Collection
from typing import List
import time

# Instantiate your client (not shown). e.g.:
# client = weaviate.connect_to_wcs(...) or
# client = weaviate.connect_to_local()

collection = client.collections.get("NameSearch")

# Get property names
property_names = list()
for p in collection.config.get().properties:
    property_names.append(p.name)

query_strings = ["Paula Merritt:"]


def filter_demo(collection: Collection, property_names: List[str], query_strings: List[str]):
    from weaviate.classes.query import Filter

    for query_string in query_strings:
        print("\n" + "=" * 40 + f"\nHits for: '{query_string}'" + "\n" + "=" * 40)
        for property_name in property_names:
            response = collection.query.fetch_objects(
                filters=Filter.by_property(property_name).equal(query_string),
            )
            if len(response.objects) > 0:
                print(f">> '{property_name}' matches")
                for obj in response.objects:
                    print(obj.properties[property_name])

t0 = time.time()
filter_demo(collection, property_names, query_strings)
t1 = time.time()
total = t1-t0
print('Time Taken')
print(total)  


Hits for: 'Paula Merritt:'
>> 'personName' matches
Paula Merritt
Time Taken
0.6340579986572266
